In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing and importing dependencies

In [ ]:
!pip install chardet
!pip install dash
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import chardet
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc
from dash import html 
from dash.dependencies import Input, Output

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('/kaggle/input/most-streamed-spotify-songs-2024/Most Streamed Spotify Songs 2024.csv', 'rb') as file:
    data = file.read(200000)
    result = chardet.detect(data)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

# Importing the dataset

In [ ]:
df = pd.read_csv('/kaggle/input/most-streamed-spotify-songs-2024/Most Streamed Spotify Songs 2024.csv',encoding="ISO-8859-1")

In [ ]:
df.head()

In [ ]:
# Shape of the data
df.shape

# Understanding the datatypes

In [ ]:
df.info()

* here datatypes are need to be changed

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['Soundcloud Streams','TIDAL Popularity'], axis=1,inplace=True)

In [ ]:
def remove(x):
    return x.replace(",","")

df["All Time Rank"]=df["All Time Rank"].apply(remove)
df["All Time Rank"]

In [ ]:
pd.to_numeric(df["All Time Rank"])

In [ ]:
def remove(x):
    return x.replace(",","")

In [ ]:
df["Release Date"]=pd.to_datetime(df["Release Date"])

In [ ]:
df = df.map(lambda x: x.replace(',','') if isinstance(x,str) else x)
df

In [ ]:
for x in df.columns[6:]:
    df[x] = pd.to_numeric(df[x],downcast="integer")
    print(df[x].dtype)

In [ ]:
df.info()
# Data types has been correctly assigned based on the columns data

# Treating Missing Values

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.dropna(subset="Artist",inplace=True)
# since 5 values are missing in artist column its better to drop these rows

In [ ]:
df["Artist"].isnull().sum()
# Null values are removed in artist column

In [ ]:
# In Spotify Streams there are 113 null values
df["Spotify Streams"].plot(kind="kde")

In [ ]:
# Since the data is right skewed we impute null values with median
df["Spotify Streams"].fillna(df["Spotify Streams"].median(),inplace=True)
df["Spotify Streams"].isnull().sum()

In [ ]:
# Null values are removed in Spotify Streams column

In [ ]:
df["Spotify Playlist Count"].plot(kind="kde")

In [ ]:
for i in df.select_dtypes(np.number):
    print(i,round(df[i].skew(),2))

In [ ]:
for i in df.columns[7:27]:
    df[i].fillna(df[i].median(),inplace=True)

In [ ]:
df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [ ]:
df.columns


# Linear Regression Model: Training and Evaluation

In [ ]:
features = ['Spotify Streams', 'All Time Rank','YouTube Views','Pandora Streams', 'TikTok Likes', 'TikTok Posts','Apple Music Playlist Count', 'Spotify Playlist Count', 'Spotify Playlist Reach','Amazon Playlist Count']
target = 'Spotify Popularity'

X = df[features]
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Linear Regression Mean Squared Error: {mse}')



# **Random Forest Regressor: Training and Evaluation**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Random Forest Regressor Mean Squared Error: {mse_rf}')


# Feature Importance Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances
importances = rf_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance Analysis')
plt.show()


# Gradient Boosting Regressor: Training and Evaluation

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=10)
gbm.fit(X_train, y_train)
y_pred_gbm = gbm.predict(X_test)

mse_gbm = mean_squared_error(y_test, y_pred_gbm)
print(f'Gradient Boosting Regressor Mean Squared Error: {mse_gbm}')


# Gradient Boosting Feature Importance Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances
importances = gbm.feature_importances_
feature_names = X.columns

# Create a DataFrame for visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance Analysis')
plt.show()


# CatBoost Regressor: Training and Evaluation

In [ ]:
from catboost import CatBoostRegressor

catboost = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, verbose=0)
catboost.fit(X_train, y_train)
y_pred_catboost = catboost.predict(X_test)

mse_catboost = mean_squared_error(y_test, y_pred_catboost)
print(f'CatBoost Regressor Mean Squared Error: {mse_catboost}')


# CatBoost Regressor: Hyperparameter Tuning with GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
}

catboost = CatBoostRegressor(verbose=0)
grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_catboost = grid_search.best_estimator_
y_pred_best_catboost = best_catboost.predict(X_test)
mse_best_catboost = mean_squared_error(y_test, y_pred_best_catboost)
print(f'Best CatBoost Regressor Mean Squared Error: {mse_best_catboost}')


# Stacking Regressor: Model Training and Evaluation

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=10)),
    ('catboost', CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, verbose=0))
]

stack = StackingRegressor(estimators=estimators, final_estimator=RidgeCV())
stack.fit(X_train, y_train)
y_pred_stack = stack.predict(X_test)

mse_stack = mean_squared_error(y_test, y_pred_stack)
print(f'Stacking Regressor Mean Squared Error: {mse_stack}')
